# Bloom filter

A bloom filter is a probabilistic data structure that is fast and small and allows to:
- determine an item is FOR SURE NOT in a set
- determine an item is MAYBE YES in a set

So in blink it is useful to have the gateway encode the list of joined nodes in a bloom filter, and then disseminate it in the beacon.
Then when a node receives a beacon, it can test if the gateway still considers that the node is joined.
If not (exact), the node leaves and goes back to scanning.
If yes (probabilistic), the node just continues associated.

We are looking for some number below 10%, ideally close to 1% of certainty.

In [258]:
import hashlib
import math
import random

# FNV-1a hash function for 64-bit integers, alternative to SHA256 which is overkill for the bloom filter
def fnv1a64(x):
    hash_ = 0xcbf29ce484222325
    for b in x.to_bytes(8, 'big'):
        hash_ ^= b
        hash_ *= 0x100000001b3
        hash_ &= 0xFFFFFFFFFFFFFFFF  # stay 64-bit
    return hash_

class BloomFilter:
    def __init__(self, m_bits=128, k_hashes=3):
        self.m = m_bits
        self.k = k_hashes
        self.bits = 0

    def _hashes_sha(self, value):
        """Return k hash indices for a given uint64_t"""
        value_bytes = value.to_bytes(8, byteorder='big')
        h1 = int.from_bytes(hashlib.sha256(b'seed1' + value_bytes).digest(), 'big')
        h2 = int.from_bytes(hashlib.sha256(b'seed2' + value_bytes).digest(), 'big')
        return [((h1 + i * h2) % self.m) for i in range(self.k)]

    def _hashes_murmur(self, value):
        """Return k hash indices for a given uint64_t using fast FNV-1a"""
        h1 = fnv1a64(value)
        h2 = fnv1a64(value ^ 0x5bd1e995)  # simple variation
        return [((h1 + i * h2) % self.m) for i in range(self.k)]

    def add(self, value):
        for idx in self._hashes_murmur(value):
            self.bits |= (1 << idx)

    def contains(self, value):
        return all((self.bits & (1 << idx)) != 0 for idx in self._hashes_murmur(value))

    def dump_bits(self):
        return bin(self.bits)[2:].zfill(self.m)
    
    def dump_hex(self):
        return hex(self.bits)[2:].zfill(self.m // 4)

# === Example Usage ===

def simulate(m=128, k=3, n=50, test_trials=1000):
    bf = BloomFilter(m_bits=m, k_hashes=k)
    true_set = set()

    node_id_list = []
    # Add n known node IDs (simulate join)
    for _ in range(n):
        node_id = random.getrandbits(64)
        true_set.add(node_id)
        bf.add(node_id)

        node_id_list.append(node_id)
    
    # print("uint64_t nodes[NODES_MAX] = { ", end="")
    # for i in range(min(len(node_id_list), 101)):
    #     if i % 4 == 0 and i != 0:
    #         print("\n    ", end="")
    #     print(f"0x{node_id_list[i]:016x}, ", end="")
    # print("};\n")

    # Check for false positives
    false_positives = 0
    for _ in range(test_trials):
        fake_id = random.getrandbits(64)
        if fake_id in true_set:
            continue  # skip true positives
        if bf.contains(fake_id):
            false_positives += 1

    rate = false_positives / test_trials
    # print(f"[n={n}, m={m}, k={k}] False positive rate: {rate:.4f}")
    # # print(f"Bit array: {bf.dump_bits()}\n")
    # print(f"Bit array (hex): {bf.dump_hex()}\n")
    return rate

# Try with 100 real nodes, 128-bit filter, 3 hashes
simulate(m=128, k=3, n=100, test_trials=10000)

simulate(m=256, k=3, n=100, test_trials=10000)

simulate(m=512, k=3, n=100, test_trials=10000)

simulate(m=512+256, k=3, n=100, test_trials=10000)

simulate(m=1024, k=3, n=100, test_trials=10000)


0.0193

In [327]:
step_m = 128
step_n = 20
k = 2
test_trials = 10000
fp_threshold = 0.05 # %

best_results = {}

for n in range(20, 101, step_n):
    found = False

    for j in range(1, 10):
        m = step_m * j
        rate = simulate(m=m, k=k, n=n, test_trials=test_trials)
        # print(f"[m={m}, n={n}] → FP rate = {rate:.4%}")

        if rate < fp_threshold:
            best_results[n] = (m, rate)
            found = True
            break  # stop at the smallest m that works

    if not found:
        best_results[n] = (None, None)

# Print summary
print(f"\n=== Smallest m per n with FP < {fp_threshold:.0%} ===")
for n in sorted(best_results):
    m, rate = best_results[n]
    if m is not None:
        print(f"n={n:3d} → min m={m:4d}, FP rate={rate:.4%}")
    else:
        print(f"n={n:3d} → no m found with FP < {fp_threshold:.0%}")



=== Smallest m per n with FP < 5% ===
n= 20 → min m= 256, FP rate=3.2500%
n= 40 → min m= 512, FP rate=3.0500%
n= 60 → min m= 640, FP rate=3.8200%
n= 80 → min m= 768, FP rate=4.2600%
n=100 → min m=1024, FP rate=4.8600%
